<a href="https://colab.research.google.com/github/BeforeTheStorm202/DACN/blob/main/InSDN_MLP_SHAP_whitelist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 15.6 MB/s eta 0:00:00


In [2]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Load dataset & model

In [4]:
df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/multi/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385156 entries, 0 to 385155
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      385156 non-null  float64
 1   Tot Fwd Pkts       385156 non-null  float64
 2   Tot Bwd Pkts       385156 non-null  float64
 3   TotLen Fwd Pkts    385156 non-null  float64
 4   TotLen Bwd Pkts    385156 non-null  float64
 5   Fwd Pkt Len Max    385156 non-null  float64
 6   Fwd Pkt Len Min    385156 non-null  float64
 7   Fwd Pkt Len Mean   385156 non-null  float64
 8   Fwd Pkt Len Std    385156 non-null  float64
 9   Bwd Pkt Len Max    385156 non-null  float64
 10  Bwd Pkt Len Min    385156 non-null  float64
 11  Bwd Pkt Len Mean   385156 non-null  float64
 12  Bwd Pkt Len Std    385156 non-null  float64
 13  Flow Byts/s        385156 non-null  float64
 14  Flow Pkts/s        385156 non-null  float64
 15  Flow IAT Mean      385156 non-null  float64
 16  Fl

In [5]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/insdn_mlp_multi.h5', compile=True)

In [6]:
x_normal = df[df['Label'] == 0].reset_index(drop=True)

In [7]:
x_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76592 entries, 0 to 76591
Data columns (total 77 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      76592 non-null  float64
 1   Tot Fwd Pkts       76592 non-null  float64
 2   Tot Bwd Pkts       76592 non-null  float64
 3   TotLen Fwd Pkts    76592 non-null  float64
 4   TotLen Bwd Pkts    76592 non-null  float64
 5   Fwd Pkt Len Max    76592 non-null  float64
 6   Fwd Pkt Len Min    76592 non-null  float64
 7   Fwd Pkt Len Mean   76592 non-null  float64
 8   Fwd Pkt Len Std    76592 non-null  float64
 9   Bwd Pkt Len Max    76592 non-null  float64
 10  Bwd Pkt Len Min    76592 non-null  float64
 11  Bwd Pkt Len Mean   76592 non-null  float64
 12  Bwd Pkt Len Std    76592 non-null  float64
 13  Flow Byts/s        76592 non-null  float64
 14  Flow Pkts/s        76592 non-null  float64
 15  Flow IAT Mean      76592 non-null  float64
 16  Flow IAT Std       765

In [8]:
x_normal['Label'].value_counts()

0    76592
Name: Label, dtype: int64

In [9]:
df = df.drop(columns=['Label'])

In [10]:
x_normal = x_normal.drop(columns=['Label'])

In [38]:
feature_list = x_normal.columns

# SHAP

In [11]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100))

In [12]:
normal_1500 = x_normal.head(1500)

In [13]:
shap_values = explainer.shap_values(normal_1500)

  0%|          | 0/1500 [00:00<?, ?it/s]

In [34]:
data_df = pd.DataFrame(shap_values[0])

In [35]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_HSJ/1500_samples_[0].csv', index=False)

In [39]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.005324,-0.004967,0.004330,0.000879,-0.003071,0.001779,0.027115,0.019613,-0.003975,-0.003574,...,0.002282,0.0,0.002213,-0.002338,-0.006176,0.002351,-0.008196,-0.004187,0.006137,-0.003489
1,-0.002830,0.005301,-0.001199,0.000000,-0.001170,-0.001958,-0.001045,-0.002484,-0.002926,-0.001644,...,0.002617,0.0,0.000000,0.000000,0.000000,0.000993,0.003167,-0.002245,0.000000,-0.002689
2,0.003506,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.002892,0.000000,0.006891,0.000000,0.011819,0.000000,0.010661,0.000000
3,0.000000,0.000000,0.000000,0.001801,0.000000,0.000000,0.000000,0.000000,0.000000,0.002624,...,0.000000,0.0,0.000000,0.002906,0.003095,0.000156,0.002264,0.000000,0.003021,0.000000
4,0.009415,0.000000,0.000363,0.000000,0.004775,0.007160,0.025359,0.021115,-0.006812,0.002334,...,0.000000,0.0,0.000000,0.000000,-0.007340,0.000000,0.000000,0.004586,0.003858,0.000480


In [55]:
data_dict = {}
for idx in range(0, 1500):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Normal sample ' + str(idx)] = row_values

In [56]:
print(data_dict)

{'Normal sample 0': ['SYN Flag Cnt', 'Init Fwd Win Byts', 'FIN Flag Cnt', 'Pkt Len Min', 'Pkt Size Avg', 'ACK Flag Cnt', 'Flow Pkts/s', 'Pkt Len Mean', 'Bwd Pkt Len Min', 'Pkt Len Max'], 'Normal sample 1': ['Init Fwd Win Byts', 'FIN Flag Cnt', 'Tot Fwd Pkts', 'Down/Up Ratio', 'Bwd IAT Tot', 'Bwd IAT Max', 'Idle Mean', 'Bwd IAT Std', 'Fwd Act Data Pkts', 'Pkt Len Std'], 'Normal sample 2': ['Down/Up Ratio', 'Init Fwd Win Byts', 'SYN Flag Cnt', 'ACK Flag Cnt', 'FIN Flag Cnt', 'Idle Mean', 'Idle Max', 'PSH Flag Cnt', 'Active Max', 'URG Flag Cnt'], 'Normal sample 3': ['Down/Up Ratio', 'Init Fwd Win Byts', 'SYN Flag Cnt', 'FIN Flag Cnt', 'ACK Flag Cnt', 'PSH Flag Cnt', 'Bwd Pkts/s', 'Flow IAT Max', 'Subflow Bwd Byts', 'Active Max'], 'Normal sample 4': ['SYN Flag Cnt', 'Init Fwd Win Byts', 'Pkt Size Avg', 'FIN Flag Cnt', 'Pkt Len Min', 'Pkt Len Mean', 'Bwd Pkt Len Min', 'ACK Flag Cnt', 'Pkt Len Max', 'Flow Pkts/s'], 'Normal sample 5': ['Down/Up Ratio', 'Init Fwd Win Byts', 'FIN Flag Cnt', 'Pk

In [57]:
len(data_dict)

1500

# Extract local feature for each adv sample

In [ ]:
for idx in range(0, 76592):
  # Create a DataFrame with the SHAP values and feature names
  df_shap_values = pd.DataFrame({'SHAP Values': exp[idx].values}, index=feature_list)
  df_shap_values.sort_values(by=['SHAP Values'], ascending=False, inplace=True)
  df_shap_values.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/Normal_local/SHAP_Samples_' + str(idx) + '.csv')
  print('Extracted local importance feature of the ' + str(idx) + ' sample.')

# Detect normal samples

In [72]:
whitelist = freq_df.iloc[:, 0][:38].tolist()
print(whitelist)

['FIN Flag Cnt', 'SYN Flag Cnt', 'Init Fwd Win Byts', 'Flow Pkts/s', 'ACK Flag Cnt', 'Pkt Size Avg', 'Bwd Pkts/s', 'Down/Up Ratio', 'Pkt Len Mean', 'Pkt Len Min', 'Bwd Pkt Len Min', 'Pkt Len Max', 'Init Bwd Win Byts', 'Fwd Pkt Len Max', 'Fwd IAT Tot', 'Flow IAT Max', 'PSH Flag Cnt', 'Fwd Pkt Len Std', 'Fwd Pkt Len Mean', 'Pkt Len Std', 'Bwd IAT Tot', 'Idle Max', 'Fwd Seg Size Avg', 'Fwd IAT Std', 'Bwd PSH Flags', 'Flow Duration', 'Bwd IAT Std', 'Fwd Pkt Len Min', 'Flow IAT Std', 'Bwd IAT Max', 'Idle Mean', 'Fwd IAT Max', 'URG Flag Cnt', 'Idle Min', 'Flow IAT Mean', 'Fwd IAT Mean', 'Flow Byts/s', 'Bwd Pkt Len Mean']


In [73]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:511


In [74]:
whitelist = freq_df.iloc[:, 0][:24].tolist()
print(whitelist)

['FIN Flag Cnt', 'SYN Flag Cnt', 'Init Fwd Win Byts', 'Flow Pkts/s', 'ACK Flag Cnt', 'Pkt Size Avg', 'Bwd Pkts/s', 'Down/Up Ratio', 'Pkt Len Mean', 'Pkt Len Min', 'Bwd Pkt Len Min', 'Pkt Len Max', 'Init Bwd Win Byts', 'Fwd Pkt Len Max', 'Fwd IAT Tot', 'Flow IAT Max', 'PSH Flag Cnt', 'Fwd Pkt Len Std', 'Fwd Pkt Len Mean', 'Pkt Len Std', 'Bwd IAT Tot', 'Idle Max', 'Fwd Seg Size Avg', 'Fwd IAT Std']


In [75]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:936


## False positive of XAI on normal samples of trainset:
top 38: 511/1500

top 24: 936/1500

# New whitelist: 38 top features with over 10k freq each

In [58]:
df = pd.DataFrame.from_dict(data_dict, orient='index', columns=['Feature 1', 'Feature 2', 'Feature 3', 'Feature 4', 'Feature 5', 'Feature 6', 'Feature 7', 'Feature 8', 'Feature 9', 'Feature 10'])

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, Normal sample 0 to Normal sample 1499
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Feature 1   1500 non-null   object
 1   Feature 2   1500 non-null   object
 2   Feature 3   1500 non-null   object
 3   Feature 4   1500 non-null   object
 4   Feature 5   1500 non-null   object
 5   Feature 6   1500 non-null   object
 6   Feature 7   1500 non-null   object
 7   Feature 8   1500 non-null   object
 8   Feature 9   1500 non-null   object
 9   Feature 10  1500 non-null   object
dtypes: object(10)
memory usage: 128.9+ KB


In [60]:
df.head()

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10
Normal sample 0,SYN Flag Cnt,Init Fwd Win Byts,FIN Flag Cnt,Pkt Len Min,Pkt Size Avg,ACK Flag Cnt,Flow Pkts/s,Pkt Len Mean,Bwd Pkt Len Min,Pkt Len Max
Normal sample 1,Init Fwd Win Byts,FIN Flag Cnt,Tot Fwd Pkts,Down/Up Ratio,Bwd IAT Tot,Bwd IAT Max,Idle Mean,Bwd IAT Std,Fwd Act Data Pkts,Pkt Len Std
Normal sample 2,Down/Up Ratio,Init Fwd Win Byts,SYN Flag Cnt,ACK Flag Cnt,FIN Flag Cnt,Idle Mean,Idle Max,PSH Flag Cnt,Active Max,URG Flag Cnt
Normal sample 3,Down/Up Ratio,Init Fwd Win Byts,SYN Flag Cnt,FIN Flag Cnt,ACK Flag Cnt,PSH Flag Cnt,Bwd Pkts/s,Flow IAT Max,Subflow Bwd Byts,Active Max
Normal sample 4,SYN Flag Cnt,Init Fwd Win Byts,Pkt Size Avg,FIN Flag Cnt,Pkt Len Min,Pkt Len Mean,Bwd Pkt Len Min,ACK Flag Cnt,Pkt Len Max,Flow Pkts/s


In [61]:
df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_top10_of_each_normalfile.csv')

In [62]:
freq = {}
for column in df.columns:
  for feature in df[column]:
    if feature in freq:
      continue
    else:
      count = sum(df[column_2] == feature for column_2 in df.columns)
      freq[feature] = count.sum()
      print(freq[feature])

1196
986
732
148
767
473
56
582
275
283
131
238
588
197
180
1223
667
584
808
156
188
884
752
197
193
37
188
182
25
132
81
260
65
123
44
64
30
153
156
23
77
21
35
136
40
34
30
43
34
22
36
23
39
31
38
34
47
30
51
35
38
20
27
32


In [67]:
len(freq)

64

In [63]:
freq_df = pd.DataFrame(list(freq.items()), columns=['Feature', 'Frequency'])
freq_df.head()

,Feature,Frequency
0,SYN Flag Cnt,1196
1,Init Fwd Win Byts,986
2,Down/Up Ratio,732
3,Fwd Pkt Len Min,148
4,Pkt Size Avg,767


In [64]:
freq_df.sort_values(by=['Frequency'], ascending=False, inplace=True)
freq_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 15 to 61
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Feature    64 non-null     object
 1   Frequency  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ KB


In [65]:
freq_df.head(24)

,Feature,Frequency
15,FIN Flag Cnt,1223
0,SYN Flag Cnt,1196
1,Init Fwd Win Byts,986
21,Flow Pkts/s,884
18,ACK Flag Cnt,808
4,Pkt Size Avg,767
22,Bwd Pkts/s,752
2,Down/Up Ratio,732
16,Pkt Len Mean,667
12,Pkt Len Min,588


In [66]:
freq_df[:38].to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_top24_InSDN.csv', index=False)